In [1]:
# =========================================================
# Export Predicted Student Performance for Power BI
# =========================================================

# (Chạy cell này đầu tiên)
import pandas as pd
import numpy as np
import joblib
import os


In [2]:
# =========================================================
#  Đặt working directory về thư mục chứa file Notebook
# =========================================================

# Đảm bảo luôn đọc được file CSV & PKL cùng cấp
os.chdir(os.path.dirname(os.path.abspath("__file__")))

# Kiểm tra xem file có tồn tại không
for file_name in ["student_performance_dataset1.csv", "model_randomforest.pkl"]:
    if not os.path.exists(file_name):
        print(f"⚠️ Không tìm thấy file: {file_name}")
    else:
        print(f"✅ Tìm thấy file: {file_name}")


✅ Tìm thấy file: student_performance_dataset1.csv
✅ Tìm thấy file: model_randomforest.pkl


In [3]:
# =========================================================
# Load dữ liệu và mô hình Random Forest
# =========================================================
data = pd.read_csv("student_performance_dataset1.csv")
model = joblib.load("model_randomforest.pkl")

print(" Loaded data:", data.shape)
print(" Loaded model: Random Forest")
data.head()


 Loaded data: (14003, 16)
 Loaded model: Random Forest


,StudyHours,Attendance,Resources,Extracurricular,Motivation,Internet,Gender,Age,LearningStyle,OnlineCourses,Discussions,AssignmentCompletion,ExamScore,EduTech,StressLevel,FinalGrade
0,19,64,1,0,0,1,0,19,2,8,1,59,40,0,1,3
1,19,64,1,0,0,1,0,23,3,16,0,90,66,0,1,2
2,19,64,1,0,0,1,0,28,1,19,0,67,99,1,1,0
3,19,64,1,1,0,1,0,19,2,8,1,59,40,0,1,3
4,19,64,1,1,0,1,0,23,3,16,0,90,66,0,1,2


In [4]:
# =========================================================
#  Xác định đặc trưng đầu vào dùng cho dự đoán
# =========================================================
selected_features = [
    'Discussions', 'StressLevel', 'AssignmentCompletion',
    'Attendance', 'LearningStyle', 'EduTech', 'StudyHours'
]

X = data[selected_features]
X.head()


,Discussions,StressLevel,AssignmentCompletion,Attendance,LearningStyle,EduTech,StudyHours
0,1,1,59,64,2,0,19
1,0,1,90,64,3,0,19
2,0,1,67,64,1,1,19
3,1,1,59,64,2,0,19
4,0,1,90,64,3,0,19


In [8]:
# =========================================================
# Thực hiện dự đoán
# =========================================================
preds = model.predict(X)
pred_labels = np.round(preds).astype(int)

print("✅ Dự đoán thành công! 5 giá trị đầu tiên:")
print(preds[:10])


✅ Dự đoán thành công! 5 giá trị đầu tiên:
[2.97 1.98 0.   2.97 1.98 0.   2.97 1.98 0.   2.97]


In [9]:
# =========================================================
# Gán nhãn Pred_Grade dựa theo giá trị dự đoán
# =========================================================
grade_map = {0: "A", 1: "B", 2: "C", 3: "D"}
data["Pred_Grade"] = [grade_map.get(x, "Unknown") for x in pred_labels]

data[["FinalGrade", "Pred_Grade"]].head(10)



,FinalGrade,Pred_Grade
0,3,D
1,2,C
2,0,A
3,3,D
4,2,C
5,0,A
6,3,D
7,2,C
8,0,A
9,3,D


In [13]:
# =========================================================
# Gán mức rủi ro học tập dựa theo Pred_Grade
# =========================================================
def risk_label(grade):
    if grade in ["A", "B"]:
        return "Ổn định"
    elif grade == "C":
        return "Cần hỗ trợ"
    elif grade == "D":
        return "Nguy cơ cao"
    else:
        return "Không xác định"

data["Risk_Level"] = data["Pred_Grade"].apply(risk_label)
data[["Pred_Grade", "Risk_Level"]].head(10)


,Pred_Grade,Risk_Level
0,D,Nguy cơ cao
1,C,Cần hỗ trợ
2,A,Ổn định
3,D,Nguy cơ cao
4,C,Cần hỗ trợ
5,A,Ổn định
6,D,Nguy cơ cao
7,C,Cần hỗ trợ
8,A,Ổn định
9,D,Nguy cơ cao


In [14]:
data.tail()

,StudyHours,Attendance,Resources,Extracurricular,Motivation,Internet,Gender,Age,LearningStyle,OnlineCourses,Discussions,AssignmentCompletion,ExamScore,EduTech,StressLevel,FinalGrade,Pred_Grade,Risk_Level
13998,30,62,0,1,1,1,0,22,2,2,1,100,71,1,2,1,B,Ổn định
13999,30,62,0,1,1,1,0,23,3,12,1,72,55,1,1,2,C,Cần hỗ trợ
14000,22,90,2,0,1,1,0,23,3,0,1,80,56,0,0,2,C,Cần hỗ trợ
14001,22,90,2,0,1,1,0,29,2,16,0,50,62,1,2,2,C,Cần hỗ trợ
14002,10,86,2,1,2,1,0,18,2,8,1,66,77,1,2,1,B,Ổn định


In [15]:
# =========================================================
# Xuất file kết quả dự đoán để dùng trong Power BI
# =========================================================
output_path = "student_performance_predicted.csv"
data.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f" Exported successfully: {output_path}")


 Exported successfully: student_performance_predicted.csv
